In [1]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert

data_path = '../data/all_sample_20220821_spark.csv'
all_data = pd.read_csv(data_path)



/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_39706/1986276430.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv(data_path)


# 需要过虑
* time_list是单个的时间，需要确定时间的下限，不能是1900年
* time_pair_list是start和end的时间pair，需要过滤掉太长的时间差

In [33]:
from datetime import datetime

def parse_ym(s):
    year_s, mon_s = s.split('-')
    return datetime(int(year_s), int(mon_s), 1)

time_list = []
time_pair_list = []

for i in range(len(all_data)):
    try:
        jobtrack = all_data['jobTracks'][i]
        jobtrack = jobtrack.replace("\'", "")
        jobtrack_list_dict = json.loads(jobtrack)

        for dic in jobtrack_list_dict:
            startDate = dic.get('startDate', None)
            endDate = dic.get('endDate', None)

            if not startDate or not endDate:
                continue

            start = parse_ym(startDate)
            end = parse_ym(endDate)

            # 这个down_limit该如何定呢？
            down_limit = parse_ym('1990-01')
            up_limit = datetime.now()

            if end > up_limit or start < down_limit: 
                continue

            if end <= start:
                continue

            time_list.append(startDate)
            time_list.append(endDate)

            
            delta = (end - start).days
            # if delta > 3650:
            #     continue
            time_pair_list.append((startDate, endDate, delta))
    except:
        pass


delta_days_list = [days for _, _, days in time_pair_list]


percent_list = list(range(1, 10)) 
percent_list.extend([9.5, 9.6, 9.7, 9.8, 9.9, 10])
percent_list = map(lambda x: x*10, percent_list)

# 计算分位点
for percent in percent_list:
    num = np.percentile(delta_days_list, percent, interpolation='midpoint')
    print("%{}的分位点是：{}".format(percent, num))

%10的分位点是：212.0
%20的分位点是：335.0
%30的分位点是：426.0
%40的分位点是：549.0
%50的分位点是：700.0
%60的分位点是：821.0
%70的分位点是：1034.0
%80的分位点是：1248.0
%90的分位点是：1705.0
%95.0的分位点是：2192.0
%96.0的分位点是：2404.0
%97.0的分位点是：2619.0
%98.0的分位点是：2983.0
%99.0的分位点是：3653.0
%100的分位点是：11778.0


In [24]:
time_list

with open('time_list', "w") as fo:
    for t in time_list:
        fo.write(str(t) + '\n')

### 查看每个时间的重复次数。有些时间重复次数太少，比如1980。确定一个时间下限。

In [29]:
cat time_list | sort | uniq -c | sort -k1 -rn

13525 2022-04
10619 2022-05
10333 2022-03
8169 2022-06
6734 2022-02
6267 2018-07
6165 2022-01
5829 2021-03
5723 2018-06
5648 2018-03
5636 2021-04
5612 2019-07
5359 2021-01
5340 2019-01
5323 2021-06
5320 2017-07
5301 2018-01
5223 2019-03
5186 2020-07
5129 2018-04
5102 2018-08
5090 2018-05
5026 2021-07
5016 2020-06
5004 2019-06
4993 2020-03
4971 2021-05
4942 2020-01
4929 2017-03
4923 2016-07
4830 2020-09
4826 2019-04
4794 2021-12
4787 2019-05
4736 2020-04
4691 2018-09
4509 2015-07
4507 2020-08
4503 2019-08
4431 2016-06
4407 2021-08
4350 2017-06
4335 2022-07
4321 2016-03
4298 2020-12
4291 2020-05
4281 2017-01
4254 2021-09
4242 2019-09
4234 2020-10
4191 2018-10
4040 2014-07
3975 2017-09
3971 2017-08
3937 2016-01
3914 2019-10
3899 2017-05
3790 2017-04
3771 2016-08
3730 2018-12
3711 2016-05
3662 2019-12
3629 2015-01
3610 2020-11
3580 2015-05
3573 2015-06
3511 2017-12
3431 2015-03
3425 2017-02
3368 2016-09
3303 2021-10
3296 2016-04
3287 2015-08
3276 2019-11
3273 2021-11
3273 2017-10
3217 2018

In [35]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert


def load_csv_data(data_path):
    '''
    读取csv文件
    '''
    df = pd.read_csv(data_path)
    return df

if __name__ == "__main__":
    print("running...")

    data_path = '../data/all_sample_20220821_spark.csv'
    all_data = load_csv_data(data_path)

running...


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_39706/2919539378.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [42]:
from datetime import datetime

def parse_ym(s):
    year_s, mon_s = s.split('-')
    return datetime(int(year_s), int(mon_s), 1)

def get_job_time_fun(series, col):
    '''
    从jobTracks列提取工作的开始时间、结束时间和时间差(天数)
    '''
    time_pair_list = []
    try:
        jobtrack = series[col]
        jobtrack = jobtrack.replace("\'", "")
        jobtrack_list_dict = json.loads(jobtrack)

        for dic in jobtrack_list_dict:
            startDate = dic.get('startDate', None)
            endDate = dic.get('endDate', None)
        
            if not startDate or not endDate:
                continue

            start = parse_ym(startDate)
            end = parse_ym(endDate)

            # 这个down_limit该如何定呢？
            down_limit = parse_ym('1990-01')
            up_limit = datetime.now()

            if end > up_limit or start < down_limit: 
                continue

            if end <= start:
                continue

            delta = (end - start).days

            time_pair_list.append((startDate, endDate, delta))
        
        return time_pair_list
    except:
        return []

def work_duration_fun(series, col):
    job_time_delta = series[col]
    time_delta = [days for _,_,days in job_time_delta]

    return np.mean(time_delta)

all_data['job_time_delta'] = all_data.apply(get_job_time_fun, axis=1, args=('jobTracks', ))
all_data['work_duration_mean'] = all_data.apply(work_duration_fun, axis=1, args=('job_time_delta', ))

/Users/jinrirencai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jinrirencai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [48]:
all_data[['job_time_delta', 'work_duration_mean']]['job_time_delta'][3]

[('2021-09', '2021-11', 61), ('2016-06', '2016-09', 92)]

In [45]:
all_data[['job_time_delta', 'work_duration_mean']]

,job_time_delta,work_duration_mean
0,"[(2020-02, 2021-12, 669), (2013-08, 2019-09, 2...",1209.500000
1,"[(2020-06, 2021-09, 457), (2019-03, 2020-06, 4...",506.000000
2,"[(2020-12, 2021-10, 304), (2016-11, 2020-11, 1...",811.500000
3,"[(2021-09, 2021-11, 61), (2016-06, 2016-09, 92)]",76.500000
4,"[(2016-07, 2019-05, 1034)]",1034.000000
...,...,...
81291,"[(2022-04, 2022-07, 91), (2022-04, 2022-07, 91)]",91.000000
81292,"[(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...",1582.750000
81293,"[(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...",1582.750000
81294,"[(2018-04, 2020-07, 822), (2016-11, 2018-03, 4...",598.666667


In [49]:
all_data.iloc[3]

Unnamed: 0                                                            3
cv_id                                  2c9207157d308592017d4a94cb1113a1
jd_id                                  2c9207157d2f0ec5017d5a9265ba33d9
jd_code                                          JD20211126124504034196
title                                                     海外区域运营经理/高级经理
                                            ...                        
projectTracks         ['{"description": "主播表现，直播平均 2 场，平台新用户 12926，主...
ds.1                                                           20220821
job_time_delta         [(2021-09, 2021-11, 61), (2016-06, 2016-09, 92)]
work_duration                                                      76.5
work_duration_mean                                                 76.5
Name: 3, Length: 90, dtype: object

In [61]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert


def load_csv_data(data_path):
    '''
    读取csv文件
    '''
    df = pd.read_csv(data_path)
    return df

from datetime import datetime

def parse_ym(s):
    year_s, mon_s = s.split('-')
    return datetime(int(year_s), int(mon_s), 1)

def get_job_time_fun(series, col):
    '''
    从jobTracks列提取工作的开始时间、结束时间和时间差(天数)
    '''
    time_pair_list = []
    try:
        jobtrack = series[col]
        jobtrack = jobtrack.replace("\'", "")
        jobtrack_list_dict = json.loads(jobtrack)

        for dic in jobtrack_list_dict:
            startDate = dic.get('startDate', None)
            endDate = dic.get('endDate', None)
        
            if not startDate or not endDate:
                continue

            start = parse_ym(startDate)
            end = parse_ym(endDate)

            # 这个down_limit该如何定呢？
            down_limit = parse_ym('1990-01')
            up_limit = datetime.now()

            if end > up_limit or start < down_limit: 
                continue

            if end <= start:
                continue

            delta = (end - start).days

            time_pair_list.append((startDate, endDate, delta))
        
        return time_pair_list
    except:
        return []

def work_duration_mean_fun(series, col):
    '''
    对各份工作时长取均值
    '''
    job_time_delta = series[col]
    time_delta = [days for _,_,days in job_time_delta]

    return np.mean(time_delta)

def get_work_duration_mean(data_path):
    '''
    给定数据路径，得到work_duration_mean列
    '''
    all_data = load_csv_data(data_path)
    all_data['job_time_delta'] = all_data.apply(get_job_time_fun, axis=1, args=('jobTracks', ))
    all_data['work_duration_mean'] = all_data.apply(work_duration_mean_fun, axis=1, args=('job_time_delta', ))
    return all_data


if __name__ == "__main__":
    print("running...")

    data_path = '../data/all_sample_20220821_spark.csv'
    all_data = get_work_duration_mean(data_path)
    
    print(all_data[['job_time_delta', 'work_duration_mean']])


running...


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_39706/933454647.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)
/Users/jinrirencai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jinrirencai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                                          job_time_delta  work_duration_mean
0      [(2020-02, 2021-12, 669), (2013-08, 2019-09, 2...         1209.500000
1      [(2020-06, 2021-09, 457), (2019-03, 2020-06, 4...          506.000000
2      [(2020-12, 2021-10, 304), (2016-11, 2020-11, 1...          811.500000
3       [(2021-09, 2021-11, 61), (2016-06, 2016-09, 92)]           76.500000
4                             [(2016-07, 2019-05, 1034)]         1034.000000
...                                                  ...                 ...
81291   [(2022-04, 2022-07, 91), (2022-04, 2022-07, 91)]           91.000000
81292  [(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...         1582.750000
81293  [(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...         1582.750000
81294  [(2018-04, 2020-07, 822), (2016-11, 2018-03, 4...          598.666667
81295  [(2022-02, 2022-08, 181), (2019-04, 2022-01, 1...          814.000000

[81296 rows x 2 columns]


In [62]:
all_data[['job_time_delta', 'work_duration_mean']]

,job_time_delta,work_duration_mean
0,"[(2020-02, 2021-12, 669), (2013-08, 2019-09, 2...",1209.500000
1,"[(2020-06, 2021-09, 457), (2019-03, 2020-06, 4...",506.000000
2,"[(2020-12, 2021-10, 304), (2016-11, 2020-11, 1...",811.500000
3,"[(2021-09, 2021-11, 61), (2016-06, 2016-09, 92)]",76.500000
4,"[(2016-07, 2019-05, 1034)]",1034.000000
...,...,...
81291,"[(2022-04, 2022-07, 91), (2022-04, 2022-07, 91)]",91.000000
81292,"[(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...",1582.750000
81293,"[(2020-08, 2022-07, 699), (2017-01, 2020-07, 1...",1582.750000
81294,"[(2018-04, 2020-07, 822), (2016-11, 2018-03, 4...",598.666667


In [70]:
all_data[['job_time_delta', 'work_duration_mean']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   job_time_delta      81296 non-null  object 
 1   work_duration_mean  76290 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.2+ MB


In [67]:
for index, item in enumerate(all_data['job_time_delta']):
    if not item:
        print(index, item)

11 []
12 []
16 []
17 []
18 []
21 []
84 []
98 []
177 []
179 []
215 []
267 []
291 []
292 []
295 []
332 []
333 []
341 []
357 []
358 []
359 []
360 []
403 []
473 []
496 []
504 []
519 []
545 []
579 []
580 []
581 []
602 []
629 []
642 []
644 []
645 []
698 []
701 []
711 []
714 []
753 []
763 []
781 []
814 []
818 []
824 []
852 []
853 []
903 []
931 []
935 []
936 []
937 []
938 []
981 []
1033 []
1056 []
1057 []
1058 []
1091 []
1106 []
1139 []
1140 []
1167 []
1172 []
1206 []
1210 []
1213 []
1221 []
1227 []
1229 []
1235 []
1276 []
1303 []
1305 []
1333 []
1370 []
1371 []
1372 []
1373 []
1374 []
1375 []
1376 []
1377 []
1386 []
1432 []
1476 []
1477 []
1513 []
1526 []
1566 []
1569 []
1576 []
1590 []
1597 []
1606 []
1607 []
1621 []
1622 []
1623 []
1625 []
1672 []
1726 []
1789 []
1799 []
1800 []
1823 []
1877 []
1878 []
1879 []
1880 []
1881 []
1888 []
1894 []
1895 []
1912 []
1930 []
1931 []
1935 []
1937 []
1946 []
1952 []
2029 []
2095 []
2118 []
2125 []
2262 []
2275 []
2277 []
2278 []
2279 []
2284 []
2312 []